<a href="https://colab.research.google.com/github/nicolasmaibarra-code/actuarial_pricing_ML/blob/main/pricing_actuarial_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!git

Cloning into 'credit_card_fraud'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 15 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 5.51 KiB | 2.76 MiB/s, done.
Resolving deltas: 100% (3/3), done.


# Introdución
**Modelo de Pricing Dinámico de Seguros con Machine Learning**

La idea es construir un sistema que calcule la prima "justa" para un asegurado usando ML en vez de las tablas tradicionales, y comparar ambos enfoques. Esto es exactamente lo que están haciendo las insurtechs hoy en día en Argentina y el mundo.

Lo que lo hace llamativo es que toca varios mundos a la vez: actuaría tradicional (tablas, GLMs, frecuencia-severidad), machine learning aplicado (XGBoost, Random Forest), y un problema de negocio real con impacto económico directo. No es un proyecto de notebook genérico, es algo que cualquier aseguradora o insurtech contrataría a alguien para hacer.

La estructura podría ser así: primero modelás frecuencia y severidad de siniestros por separado con modelos clásicos actuariales como GLM con distribución Poisson y Gamma, que es el estándar de la industria.

Luego replicás lo mismo con modelos de ML y comparás resultados. Finalmente agregás una capa de interpretabilidad con SHAP values para explicar qué variables más influyen en el riesgo, que es algo que los reguladores como la SSN exigen en la práctica.


In [ ]:
from sklearn.datasets import fetch_openml

# Frecuencia
freMTPL2freq = fetch_openml(data_id=41214, as_frame=True).frame

# Severidad, el importe de los siniestro de cada cuota
freMTPL2sev = fetch_openml(data_id=41215, as_frame=True).frame


# Modelos actuariales clasicos. GLM con Poisson para la frecuencia de siniestros

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf


# 1. Definir el modelo GLM con familia Poisson
# Usamos exposure como offset (log del tiempo de exposición)
model = smf.glm(
    formula="ClaimNb ~ VehAge + DrivAge ",
    data=freMTPL2freq,
    family=sm.families.Poisson(),
    offset=np.log(freMTPL2freq["Exposure"])
)

# 2. Ajustar el modelo
result = model.fit()

# 3. Resumen de resultados
print(result.summary())

# 4. Predicciones
df=pd.DataFrame()
df["IDpol"] = freMTPL2freq["IDpol"]
df["exposure"]=freMTPL2freq["Exposure"]
df["claims"]=freMTPL2freq["ClaimNb"]
df["predicted_claims"] = result.predict(freMTPL2freq)
print(df)


                 Generalized Linear Model Regression Results                  
Dep. Variable:                ClaimNb   No. Observations:               678013
Model:                            GLM   Df Residuals:                   678010
Model Family:                 Poisson   Df Model:                            2
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4599e+05
Date:                Thu, 26 Feb 2026   Deviance:                   2.2266e+05
Time:                        20:02:31   Pearson chi2:                 1.84e+06
No. Iterations:                     7   Pseudo R-squ. (CS):           0.002675
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.7668      0.019    -90.951      0.0

# Modelos actuariales clasicos. GLM con Gamma para el monto de siniestros

In [ ]:
from scipy.special import gamma
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf


# 1. Definir el modelo GLM con familia Poisson
# Usamos exposure como offset (log del tiempo de exposición)
# Severidad (Gamma)

sev_enriched = freMTPL2sev.merge(freMTPL2freq[["IDpol", "VehAge", "DrivAge", "BonusMalus"]],
                            on="IDpol", how="left" )

glm_sev = smf.glm( formula="ClaimAmount ~ VehAge + DrivAge + BonusMalus",
                  data=sev_enriched,
                  family=sm.families.Gamma(sm.families.links.log()) ).fit()
# 2. Ajustar el modelo
result_sev = glm_sev.model.fit()

# 3. Resumen de resultados
print(result_sev.summary())

# 4. Predicciones
df_sev=pd.DataFrame()
df_sev["IDpol"] = freMTPL2sev["IDpol"]
df_sev["predicted_importes"] = result_sev.predict(sev_enriched)
print(df_sev)

# Combinar resultados (ejemplo: promedio de severidad por póliza)
sev_summary = df_sev.groupby("IDpol")["predicted_importes"].mean().reset_index()

/usr/local/lib/python3.12/dist-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:            ClaimAmount   No. Observations:                26444
Model:                            GLM   Df Residuals:                    26440
Model Family:                   Gamma   Df Model:                            3
Link Function:                    log   Scale:                          87.374
Method:                          IRLS   Log-Likelihood:            -3.0100e+05
Date:                Thu, 26 Feb 2026   Deviance:                       45057.
Time:                        20:22:04   Pearson chi2:                 2.31e+06
No. Iterations:                    23   Pseudo R-squ. (CS):          0.0006165
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.2875      0.362     20.136      0.0

In [ ]:
#Union de DF
# Merge con frecuencia
premium_df = df.merge(sev_summary, on="IDpol", how="left")
premium_df["prima_pura"] = premium_df["predicted_claims"] * premium_df["predicted_importes"]

In [ ]:
premium_df

,IDpol,exposure,claims,predicted_claims,predicted_importes,prima_pura
0,1.0,0.10000,1,0.128819,NaN,NaN
1,3.0,0.77000,1,0.128819,NaN,NaN
2,5.0,0.75000,1,0.120515,NaN,NaN
3,10.0,0.09000,1,0.134916,NaN,NaN
4,11.0,0.84000,1,0.134916,NaN,NaN
...,...,...,...,...,...,...
678008,6114326.0,0.00274,0,0.129483,NaN,NaN
678009,6114327.0,0.00274,0,0.138426,NaN,NaN
678010,6114328.0,0.00274,0,0.124928,NaN,NaN
678011,6114329.0,0.00274,0,0.125552,NaN,NaN


# Modelo de ML. Random Forest

# Modelo de ML. XGBoost